In [ ]:
import pandas as pd
import numpy as np
import pylab as plt
from sklearn.cluster import KMeans

# **Avg pooling function**

In [ ]:
def RGB(img, length):
    img = plt.imread(img)
    length = length
    list1 = []
    a = 0
    b = 0
    a1 = img.shape[0] // length
    b1 = img.shape[1] // length
            
    for i in range(length):
        for j in range(length):
            for k in range(3):
                if i != length - 1:
                    if j != length - 1:
                        img1 = img[a:a+a1,b:b+b1,k]
                        c = np.sum(img1.reshape(img1.shape[0]*img1.shape[1],)) / (img1.shape[0]*img1.shape[1])
                        
                    else:
                        img1 = img[a:a+a1,b:,k]
                        c = np.sum(img1.reshape(img1.shape[0]*img1.shape[1],)) / (img1.shape[0]*img1.shape[1])
                        
                else:
                    if j != length - 1:
                        img1 = img[a:,b:b+b1,k]
                        c = np.sum(img1.reshape(img1.shape[0]*img1.shape[1],)) / (img1.shape[0]*img1.shape[1])
                        
                    else:
                        img1 = img[a:,b:,k]
                        c = np.sum(img1.reshape(img1.shape[0]*img1.shape[1],)) / (img1.shape[0]*img1.shape[1])
                        
                list1.append(int(c))
            b += b1
        b = 0
        a += a1
        
    return list1

In [ ]:
df = pd.read_csv('../input/shopee-product-matching/train.csv')
df['image_path'] = '0'
for i in range(df.shape[0]):
    df.iloc[i,5] = '../input/shopee-product-matching/train_images/' + df.iloc[i,1]

In [ ]:
df[df['label_group'] == df.loc[47,'label_group']]

# **Displaying images**

In [ ]:
img = plt.imread(df.loc[47,'image_path'])
plt.imshow(img)

In [ ]:
img_size = 5
img = np.array(RGB(df.loc[47,'image_path'], img_size)).reshape(img_size,img_size,3)
plt.imshow(img)

In [ ]:
img = plt.imread(df.loc[15067,'image_path'])
plt.imshow(img)

In [ ]:
img_size = 5
img = np.array(RGB(df.loc[15067,'image_path'], img_size)).reshape(img_size,img_size,3)
plt.imshow(img)

# **k-means**

In [ ]:
train = pd.read_csv('../input/df-5-5/df5.csv')
train = train / 255

X = np.array(train)
kmeans = KMeans(n_clusters=20000, random_state=0).fit(X)

# **F1-score**

In [ ]:

df['kmeans'] = kmeans.labels_
df['f1'] = 0
df['TP-1'] = 0
df['FP'] = 0
for i in range(df.shape[0]):
#for i in range(3):
    a = df[df['label_group'] == df.loc[i,'label_group']] 
    b = df[df['kmeans'] == df.loc[i,'kmeans']]
    df.loc[i, 'f1'] = 2 * len(set(a.index) & set(b.index)) / (a.shape[0] + b.shape[0])
    df.loc[i, 'TP-1'] = len(set(a.index) & set(b.index)) - 1
    df.loc[i, 'FP'] = len(set(b.index) - (set(a.index) & set(b.index)) )
print(df['f1'].mean())
print()
print(df['TP-1'].sum())
print(df['FP'].sum())
print()
print(df['TP-1'].mean())
print(df['FP'].mean())

In [ ]:
df[df['FP'] > 7]

# **Second k-means**

In [ ]:
for i in range(75):
    df[f'{i}'] = train[f'{i}']

a = 21000
for i in range(20000):
    if df[df['kmeans'] == i].shape[0] > 7:
        list1 = df[df['kmeans'] == i].index.copy()
        for j in list1:
            df.loc[j, 'kmeans'] = a
            a += 1

df1 = df[df['kmeans'] > 20999]

list1 = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21',
       '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33',
       '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45',
       '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57',
       '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69',
       '70', '71', '72', '73', '74']

X = np.array(df1[list1])
kmeans = KMeans(n_clusters=2250, random_state=0).fit(X)

df1['kmeans'] = kmeans.labels_

df1['kmeans'] = df1['kmeans'] + 21000

df = df[df['kmeans'] < 20999]
df = df.append(df1).reset_index(drop=True)


# **F1-score**

In [ ]:
for i in range(df.shape[0]):
#for i in range(3):
    a = df[df['label_group'] == df.loc[i,'label_group']] 
    b = df[df['kmeans'] == df.loc[i,'kmeans']]
    df.loc[i, 'f1'] = 2 * len(set(a.index) & set(b.index)) / (a.shape[0] + b.shape[0])
    df.loc[i, 'TP-1'] = len(set(a.index) & set(b.index)) - 1
    df.loc[i, 'FP'] = len(set(b.index) - (set(a.index) & set(b.index)) )
print(df['f1'].mean())
print()
print(df['TP-1'].sum())
print(df['FP'].sum())
print()
print(df['TP-1'].mean())
print(df['FP'].mean())